How to fix issue with sktime: https://chatgpt.com/share/67ce998c-9284-800b-8e92-25698f57084b

In [1]:
# !pip install -r requirements.txt

In [2]:
# !pip install xgboost

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error
import joblib
import os
import pandas as pd
import numpy as np
import json
from sklearn.base import BaseEstimator, TransformerMixin
import re
from xgboost import XGBRegressor

In [4]:
# Define dataset to use
dataset_folder = "../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips"

In [5]:
use_physics = True
use_ml = True
NORMALIZE_DATA = True

In [6]:
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'gamma': 0.3,
}    # XGBRegressor model parameters

In [7]:
ML_MODEL_USES_CALCULATED_COLUMNS = False
HYBRYD_MODEL = True    # Only hybrid model is being used

In [8]:
required_columns = [
    "Time [s]", "SoC_0", "Q_rated", "Battery Voltage [V]", 
    "Battery Current [A]", 
    "Battery Temperature [°C]", 
    "Time_difference", 
    "SoC [%]",
    "Delta"
]    # always the target variable should be the last element of this list

calculated_columns = [
    # "SoC_0", 
    "Q_rated", 
    "Time_difference",
    "SoC [%]",
    "Delta"
]

# Load and prepare data

## Define columns to use in each model

In [9]:
# get indexes of features that are relevant for the physics model
relevant_features_for_physics_model = ["SoC_0", "Battery Current [A]", "Time_difference", "Q_rated"]

feature_ids_relevant_for_physics_model = {}
for feature in relevant_features_for_physics_model:
    feature_ids_relevant_for_physics_model[feature] = required_columns.index(feature)

feature_ids_relevant_for_physics_model

{'SoC_0': 1, 'Battery Current [A]': 4, 'Time_difference': 6, 'Q_rated': 2}

In [10]:
# Define which features will be used as input for ML model (original from dataset + calculated or only original from dataset)

# get indexes of calculated_columns, and 
all_columns_indexes = list(range(len(required_columns)-1))    # all indexes. The -1 is necessary to exclude the last column, the target variable.
indexed_calculated_columns = [idx for idx, column_name in enumerate(required_columns) if column_name in calculated_columns]
indexed_calculated_columns

# according to setting, decide if all columns will be used, or only the non-calculated ones
if ML_MODEL_USES_CALCULATED_COLUMNS: 
    features_indexes_for_ml = all_columns_indexes
else:
    features_indexes_for_ml = [i for i in all_columns_indexes if i not in indexed_calculated_columns]
    
features_indexes_for_ml

[0, 1, 3, 4, 5]

## Load files as dataframes

In [11]:
def get_filepaths_with_extension(file_extension=".csv", directory="."):
    output = []
    
    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension):
            output.append(f"{directory}/{file}") 

    return output

In [12]:
# get filepaths of all train micro-trips
train_trips_filepaths = get_filepaths_with_extension(file_extension=".csv", directory=f"{dataset_folder}/train")
train_trips_filepaths[:5]

['../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA01.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA02.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA03.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA04.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA05.csv']

In [13]:
# get filepaths of all dev micro-trips
val_trips_filepaths = get_filepaths_with_extension(file_extension=".csv", directory=f"{dataset_folder}/dev")
val_trips_filepaths[:5]

['../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB19_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB20_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB21_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB22_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB23_part1.csv']

In [14]:
num_validation_files = len(val_trips_filepaths)
num_validation_files

10

In [15]:
# get filepaths of all test micro-trips
test_trips_filepaths = get_filepaths_with_extension(file_extension=".csv", directory=f"{dataset_folder}/test")
test_trips_filepaths[:5]

['../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB29_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB30_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB31_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB32_part1.csv',
 '../data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB33.csv']

In [16]:
num_test_files = len(test_trips_filepaths)
num_test_files

10

In [17]:
# load data

In [18]:
lst_train_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in train_trips_filepaths]
lst_train_dfs[0].head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,AirCon Power [kW],Heater Signal,Heater Voltage [V],Heater Current [A],Ambient Temperature [°C],Coolant Temperature Heatercore [°C],Requested Coolant Temperature [°C],Coolant Temperature Inlet [°C],Heat Exchanger Temperature [°C],Cabin Temperature Sensor [°C]
0,0.0,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.20,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
1,0.1,0.0,574.0,0.0,0.0,0.00,0.0,391.4,-2.21,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
2,0.2,0.0,574.0,0.0,0.0,-0.01,0.0,391.4,-2.26,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
3,0.3,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
4,0.4,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5


In [19]:
lst_val_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in val_trips_filepaths]
lst_val_dfs[0].head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Footweel Driver [°C],Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C]
0,0.0,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-9.70,6.0,...,4.33,3.72,4.06,4.06,4.59,6.07,3.19,3.10,3.10,3.19
1,0.1,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-9.70,6.0,...,4.33,3.72,4.06,4.07,4.61,6.09,3.19,3.10,3.11,3.21
2,0.2,0.0,512.0,0.0,0.0,-0.05,0.0,390.5,-9.70,6.0,...,4.34,3.72,4.06,4.08,4.64,6.12,3.21,3.11,3.12,3.22
3,0.3,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-10.05,6.0,...,4.35,3.73,4.06,4.09,4.66,6.15,3.23,3.12,3.14,3.24
4,0.4,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-10.40,6.0,...,4.36,3.74,4.06,4.10,4.69,6.17,3.24,3.13,3.16,3.26


In [20]:
lst_test_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in test_trips_filepaths]
lst_test_dfs[0].head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Footweel Driver [°C],Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C]
0,0.0,0.0,472.0,0.0,0.0,-0.31,0.0,366.70,-11.40,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
1,0.1,0.0,472.0,0.0,0.0,-0.32,0.0,366.70,-11.40,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
2,0.2,0.0,472.0,0.0,0.0,-0.31,0.0,366.67,-11.84,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
3,0.3,0.0,472.0,0.0,0.0,-0.28,0.0,366.62,-12.54,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
4,0.4,0.0,472.0,0.0,0.0,-0.27,0.0,366.60,-12.71,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55


## Add relevant calculated columns

In [21]:
def add_calculated_columns(df):
    battery_voltage=360
    battery_usable_capacity_kWh=18.8

    # Add initial SoC
    df["SoC_0"] = df["SoC [%]"][0]

    # Convert capacity to Ampere-seconds (As) using battery voltage
    Q_rated = (battery_usable_capacity_kWh * 1000) / battery_voltage * 3600   # Convert kWh to As

    # Add q_rated
    df["Q_rated"] = Q_rated

    # Time difference between samples
    time_difference = df['Time [s]'].diff().fillna(0)  # First diff is NaN, set to 0
    df["Time_difference"] = time_difference

    return df

In [22]:
lst_train_dfs = [add_calculated_columns(df) for df in lst_train_dfs]

In [23]:
lst_val_dfs = [add_calculated_columns(df) for df in lst_val_dfs]

In [24]:
lst_test_dfs = [add_calculated_columns(df) for df in lst_test_dfs]
lst_test_dfs[0]

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C],SoC_0,Q_rated,Time_difference
0,0.0,0.0,472.0,0.0,0.0,-0.31,0.0,366.70,-11.40,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.0
1,0.1,0.0,472.0,0.0,0.0,-0.32,0.0,366.70,-11.40,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
2,0.2,0.0,472.0,0.0,0.0,-0.31,0.0,366.67,-11.84,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
3,0.3,0.0,472.0,0.0,0.0,-0.28,0.0,366.62,-12.54,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
4,0.4,0.0,472.0,0.0,0.0,-0.27,0.0,366.60,-12.71,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,968.0,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-5.11,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9681,968.1,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-4.81,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9682,968.2,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-4.74,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9683,968.3,0.0,513.0,0.0,0.0,-0.07,0.0,344.40,-5.19,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.5,188000.0,0.1


## Add Delta

In [25]:
def coulomb_counting_on_dataframe(data):
    """
    Calculate the State of Charge (SoC) using the Coulomb Counting method.
    """
    soc_0 = data["SoC_0"]
    battery_current = data["Battery Current [A]"]
    time_difference = data["Time_difference"]
    q_rated = data["Q_rated"]

    # Coulomb Counting Method to estimate SOC
    estimated_soc = soc_0 + np.cumsum(battery_current * time_difference) / q_rated * 100
    
    return estimated_soc

In [26]:
def add_delta_to_df(df):
    df["Delta"] = df["SoC [%]"] - coulomb_counting_on_dataframe(df)
    return df

In [27]:
lst_train_dfs = [add_delta_to_df(df) for df in lst_train_dfs]
lst_val_dfs = [add_delta_to_df(df) for df in lst_val_dfs]
lst_test_dfs = [add_delta_to_df(df) for df in lst_test_dfs]

## Remove irrelevant columns

In [28]:
def remove_irrelevant_columns(df):
    return df[required_columns]

In [29]:
lst_train_dfs = [remove_irrelevant_columns(df) for df in lst_train_dfs]
lst_train_dfs[0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],Time_difference,SoC [%],Delta
0,0.0,86.9,188000.0,391.40,-2.20,21.0,0.0,86.9,0.000000
1,0.1,86.9,188000.0,391.40,-2.21,21.0,0.1,86.9,0.000118
2,0.2,86.9,188000.0,391.40,-2.26,21.0,0.1,86.9,0.000238
3,0.3,86.9,188000.0,391.40,-2.30,21.0,0.1,86.9,0.000360
4,0.4,86.9,188000.0,391.40,-2.30,21.0,0.1,86.9,0.000482
...,...,...,...,...,...,...,...,...,...
10085,1008.5,86.9,188000.0,387.91,-3.12,22.0,0.1,81.5,1.015062
10086,1008.6,86.9,188000.0,387.96,-2.37,22.0,0.1,81.5,1.015188
10087,1008.7,86.9,188000.0,388.01,-1.62,22.0,0.1,81.5,1.015274
10088,1008.8,86.9,188000.0,388.06,-0.92,22.0,0.1,81.5,1.015323


In [30]:
lst_val_dfs = [remove_irrelevant_columns(df) for df in lst_val_dfs]
lst_val_dfs[0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],Time_difference,SoC [%],Delta
0,0.0,85.8,188000.0,390.5,-9.70,6.0,0.0,85.8,0.000000
1,0.1,85.8,188000.0,390.5,-9.70,6.0,0.1,85.8,0.000516
2,0.2,85.8,188000.0,390.5,-9.70,6.0,0.1,85.8,0.001032
3,0.3,85.8,188000.0,390.5,-10.05,6.0,0.1,85.8,0.001566
4,0.4,85.8,188000.0,390.5,-10.40,6.0,0.1,85.8,0.002120
...,...,...,...,...,...,...,...,...,...
11905,1190.5,85.8,188000.0,382.8,-0.70,8.0,0.1,71.6,2.589830
11906,1190.6,85.8,188000.0,382.8,-0.70,8.0,0.1,71.6,2.589868
11907,1190.7,85.8,188000.0,382.8,-0.67,8.0,0.1,71.6,2.589903
11908,1190.8,85.8,188000.0,382.8,-0.62,8.0,0.1,71.6,2.589936


In [31]:
lst_test_dfs = [remove_irrelevant_columns(df) for df in lst_test_dfs]
lst_test_dfs[0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],Time_difference,SoC [%],Delta
0,0.0,31.5,188000.0,366.70,-11.40,11.0,0.0,31.5,0.000000
1,0.1,31.5,188000.0,366.70,-11.40,11.0,0.1,31.5,0.000606
2,0.2,31.5,188000.0,366.67,-11.84,11.0,0.1,31.5,0.001236
3,0.3,31.5,188000.0,366.62,-12.54,11.0,0.1,31.5,0.001903
4,0.4,31.5,188000.0,366.60,-12.71,11.0,0.1,31.5,0.002579
...,...,...,...,...,...,...,...,...,...
9680,968.0,31.5,188000.0,344.40,-5.11,14.0,0.1,15.4,2.983525
9681,968.1,31.5,188000.0,344.40,-4.81,14.0,0.1,15.4,2.983781
9682,968.2,31.5,188000.0,344.40,-4.74,14.0,0.1,15.4,2.984033
9683,968.3,31.5,188000.0,344.40,-5.19,14.0,0.1,15.4,2.984309


## Prepare data
Help for data preparation from ChatGPT: https://chatgpt.com/share/678e1508-2250-800b-b898-922a65ba0061

In [32]:
lst_train_dfs[0].columns

Index(['Time [s]', 'SoC_0', 'Q_rated', 'Battery Voltage [V]',
       'Battery Current [A]', 'Battery Temperature [°C]', 'Time_difference',
       'SoC [%]', 'Delta'],
      dtype='object')

In [33]:
# separate X and y from dataframe
def separate_x_and_y(df):
    """
    It outputs two dataframes, the first one contains only the X values, the second only the y values.
    """
    
    df_X = df[required_columns[:-1]]
    # df_y = df[["SoC [%]"]]
    df_y = df[["Delta"]]
    return df_X, df_y

# separate x and y in dataframes of all sets
lst_train_data = [separate_x_and_y(df) for df in lst_train_dfs]
lst_val_data = [separate_x_and_y(df) for df in lst_val_dfs]
lst_test_dfs = [separate_x_and_y(df) for df in lst_test_dfs]

In [34]:
lst_train_data[0][0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],Time_difference,SoC [%]
0,0.0,86.9,188000.0,391.40,-2.20,21.0,0.0,86.9
1,0.1,86.9,188000.0,391.40,-2.21,21.0,0.1,86.9
2,0.2,86.9,188000.0,391.40,-2.26,21.0,0.1,86.9
3,0.3,86.9,188000.0,391.40,-2.30,21.0,0.1,86.9
4,0.4,86.9,188000.0,391.40,-2.30,21.0,0.1,86.9
...,...,...,...,...,...,...,...,...
10085,1008.5,86.9,188000.0,387.91,-3.12,22.0,0.1,81.5
10086,1008.6,86.9,188000.0,387.96,-2.37,22.0,0.1,81.5
10087,1008.7,86.9,188000.0,388.01,-1.62,22.0,0.1,81.5
10088,1008.8,86.9,188000.0,388.06,-0.92,22.0,0.1,81.5


In [35]:
lst_train_data[1][0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],Time_difference,SoC [%]
0,0.0,80.3,188000.0,381.80,-37.60,23.0,0.0,80.3
1,0.1,80.3,188000.0,381.80,-37.62,23.0,0.1,80.3
2,0.2,80.3,188000.0,381.80,-37.67,23.0,0.1,80.3
3,0.3,80.3,188000.0,381.82,-37.65,23.0,0.1,80.3
4,0.4,80.3,188000.0,381.87,-37.50,23.0,0.1,80.3
...,...,...,...,...,...,...,...,...
14125,1412.5,80.3,188000.0,382.50,8.50,26.0,0.1,67.3
14126,1412.6,80.3,188000.0,382.50,8.40,26.0,0.1,67.3
14127,1412.7,80.3,188000.0,382.50,8.29,26.0,0.1,67.3
14128,1412.8,80.3,188000.0,382.50,8.14,26.0,0.1,67.3


In [36]:
# save column names before parsing to numpy arrays
X_column_names = lst_train_data[0][0].columns
X_column_names

Index(['Time [s]', 'SoC_0', 'Q_rated', 'Battery Voltage [V]',
       'Battery Current [A]', 'Battery Temperature [°C]', 'Time_difference',
       'SoC [%]'],
      dtype='object')

In [37]:
# Parse data to numpy arrays
def parse_dataframes_to_numpy_arrays(dataframes):
    # id 0 contains dataframe with X values, id 1 contains dataframe with y values.
    return dataframes[0].to_numpy(), dataframes[1].to_numpy()

# parse dataframes of all sets
lst_train_data = [parse_dataframes_to_numpy_arrays(dataframes) for dataframes in lst_train_data]
lst_val_data = [parse_dataframes_to_numpy_arrays(dataframes) for dataframes in lst_val_data]
lst_test_dfs = [parse_dataframes_to_numpy_arrays(dataframes) for dataframes in lst_test_dfs]

In [38]:
lst_train_data[0][0][0:5]

array([[ 0.000e+00,  8.690e+01,  1.880e+05,  3.914e+02, -2.200e+00,
         2.100e+01,  0.000e+00,  8.690e+01],
       [ 1.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.210e+00,
         2.100e+01,  1.000e-01,  8.690e+01],
       [ 2.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.260e+00,
         2.100e+01,  1.000e-01,  8.690e+01],
       [ 3.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.300e+00,
         2.100e+01,  1.000e-01,  8.690e+01],
       [ 4.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.300e+00,
         2.100e+01,  1.000e-01,  8.690e+01]])

In [39]:
# Data scalling

In [40]:
# create pseudo-scaler, to use it when NORMALIZE_DATA is false.
class IdentityScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        """Mimics the fit method but does nothing."""
        return self

    def transform(self, X):
        """Returns the input unchanged."""
        return np.asarray(X)  # Ensure it's a NumPy array like StandardScaler

    def inverse_transform(self, X):
        """Returns the input unchanged."""
        return np.asarray(X)

In [41]:
# test getting soc
[dataframes[0][:,-1] for dataframes in lst_train_data][0:5]

[array([86.9, 86.9, 86.9, ..., 81.5, 81.5, 81.5]),
 array([80.3, 80.3, 80.3, ..., 67.3, 67.3, 67.3]),
 array([83.5, 83.5, 83.5, ..., 75.1, 75.1, 75.1]),
 array([75.1, 75.1, 75.1, ..., 66.7, 66.7, 66.7]),
 array([66.7, 66.7, 66.7, ..., 60.2, 60.2, 60.2])]

In [42]:
lst_train_data[0][0][:,-1]

array([86.9, 86.9, 86.9, ..., 81.5, 81.5, 81.5])

In [43]:
# create Scalers, fit them and use them

scaler_X = None
scaler_y = None
scaler_soc = None

# if NORMALIZE_DATA is true, then create and train data scalers, else, scalers should be just identity functions.
if NORMALIZE_DATA:
    # Initialize scalers for features and target
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    scaler_soc = StandardScaler()
else:
    scaler_X = IdentityScaler()
    scaler_y = IdentityScaler()
    scaler_soc = IdentityScaler()

# Fit scalers on the training data only
train_features = [dataframes[0] for dataframes in lst_train_data]
train_target = [dataframes[1] for dataframes in lst_train_data]
soc_values = [dataframes[0][:,-1] for dataframes in lst_train_data]

# Combine training data to fit scalers
X_combined = np.vstack(train_features)
y_combined = np.vstack(train_target)
soc_combined = np.hstack(soc_values).reshape(-1, 1)
scaler_X.fit(X_combined)
scaler_y.fit(y_combined)
scaler_soc.fit(soc_combined)

def normalize_data(X, y):
    X = scaler_X.transform(X)
    y = scaler_y.transform(y) 
    return X, y

# Normalize training, validation, and test sets
lst_train_data = [normalize_data(data[0], data[1]) for data in lst_train_data]
lst_val_data = [normalize_data(data[0], data[1]) for data in lst_val_data]
lst_test_dfs = [normalize_data(data[0], data[1]) for data in lst_test_dfs]

In [44]:
# test scaler for soc
scaler_soc.inverse_transform(lst_train_data[0][0][:,-1].reshape(-1, 1))

array([[86.9],
       [86.9],
       [86.9],
       ...,
       [81.5],
       [81.5],
       [81.5]])

In [45]:
# Combine X data into one numpy array, same with y data
def vertically_stack_data(lst_arrays):
    all_X = [dataframes[0] for dataframes in lst_train_data]
    all_y = [dataframes[1] for dataframes in lst_train_data]
    X_combined = np.vstack(all_X)
    y_combined = np.vstack(all_y)

    return [X_combined, y_combined]

lst_train_data_by_file = lst_train_data.copy()
lst_train_data = vertically_stack_data(lst_train_data)

In [46]:
lst_train_data[0][0]

array([  -1.32002637,    0.98255685,    0.        ,    1.24208573,
          0.36720228,    0.49437417, -128.89887509,    1.44532151])

In [47]:
lst_train_data[1][0]

array([-0.84119429])

# Create Model

In [48]:
def coulomb_counting(data):
    """
    Calculate the State of Charge (SoC) using the Coulomb Counting method.
    """

    # unscale X
    data = scaler_X.inverse_transform(data)
    
    # Extract columns data
    soc_0 = data[:, feature_ids_relevant_for_physics_model["SoC_0"]]
    battery_current = data[:, feature_ids_relevant_for_physics_model["Battery Current [A]"]]
    time_difference = data[:, feature_ids_relevant_for_physics_model["Time_difference"]]
    q_rated = data[:, feature_ids_relevant_for_physics_model["Q_rated"]]


    # Coulomb Counting Method to estimate SOC
    estimated_soc = soc_0 + np.cumsum(battery_current * time_difference) / q_rated * 100
    
    return estimated_soc

# Training

In [49]:
# create directory to store results
model_name = f"""{'Physics' if use_physics else ''}{'-and-' if use_physics and use_ml else ''}{'XGBRegressor' if use_ml else ''}"""
model_name

# create folder to store results, if it does not exist yet
params_folder_name = "Data Loss + Physics Loss" if not HYBRYD_MODEL else "Delta approach"
# params += f"{', lambda='+ str(lambda_) if not HYBRYD_MODEL else ''}"   # add lambda_ if using model that is not hybrid
params_folder_name += f", learning_rate={params['learning_rate']}, max_depth={params['max_depth']}"
params_folder_name += f", subsample={params['subsample']}, colsample_bytree={params['colsample_bytree']}"
params_folder_name += f", gamma={params['gamma']}"

params_folder_name += f", Model=XGBRegressor"
params_folder_name = params_folder_name.replace(f"\n", "_")  # Replace newline with underscore
params_folder_name = re.sub(r'[<>:"/\\|?*\n]', '_', params_folder_name)  # Replace invalid characters
output_folder = f"results/{params_folder_name}/{model_name}"
os.makedirs(output_folder, exist_ok=True)

In [50]:
output_folder

'results/Delta approach, learning_rate=0.05, max_depth=5, subsample=0.6, colsample_bytree=0.6, gamma=0.3, Model=XGBRegressor/Physics-and-XGBRegressor'

In [51]:
# Save scalers
joblib.dump(scaler_X, f"{output_folder}/X-scaler.pkl")
joblib.dump(scaler_y, f"{output_folder}/y-scaler.pkl")

['results/Delta approach, learning_rate=0.05, max_depth=5, subsample=0.6, colsample_bytree=0.6, gamma=0.3, Model=XGBRegressor/Physics-and-XGBRegressor/y-scaler.pkl']

In [52]:
lst_train_data[0][:, :-2]

array([[-1.32002637,  0.98255685,  0.        ,  1.24208573,  0.36720228,
         0.49437417],
       [-1.31989889,  0.98255685,  0.        ,  1.24208573,  0.36696169,
         0.49437417],
       [-1.31977142,  0.98255685,  0.        ,  1.24208573,  0.36575878,
         0.49437417],
       ...,
       [ 0.09366603,  0.60842274,  0.        ,  0.11554667,  0.3036883 ,
        -1.60589563],
       [ 0.0937935 ,  0.60842274,  0.        ,  0.1107853 ,  0.31571746,
        -1.60589563],
       [ 0.09392098,  0.60842274,  0.        ,  0.10888076,  0.32149146,
        -1.60589563]])

In [53]:
lst_train_data[1]

array([[-0.84119429],
       [-0.84113434],
       [-0.84107304],
       ...,
       [ 0.51324914],
       [ 0.51336686],
       [ 0.51347808]])

In [54]:
# Physics and ML model.
if use_ml and use_physics:
    # train ml + physics.
    X = lst_train_data[0][:,features_indexes_for_ml]    # keep only relevant columns for ML
    y = lst_train_data[1]

    # model = XGBRegressor(n_estimators=100, learning_rate=0.1, objective='reg:squarederror', random_state=2025)
    model = XGBRegressor(**params, random_state=2025)
    model.fit(X, y)

    # save model
    model.save_model(f"{output_folder}/trained_model.json")

    # evaluation
    val_rmse = 0.0
    for X, y in lst_val_data:

        # get SoC [%]
        soc_ground_truth = X[:,-1].reshape(-1, 1)     # reshape to match shape of model output

        # get SoC from Coulomn Counting method
        coulomb_couting_output = coulomb_counting(X).reshape(-1, 1)    # reshape to match shape of model output
        
        X = X[:,features_indexes_for_ml]    # keep only relevant columns for ML

        y_pred = model.predict(X)
        
        # processing estimation
        y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
        soc_estimated = coulomb_couting_output + y_pred    # y_pred is delta
        soc_estimated = scaler_soc.transform(soc_estimated.reshape(-1, 1))
        
        val_rmse += root_mean_squared_error(soc_ground_truth, soc_estimated)
        # print(val_rmse)
        
    avg_val_rmse = val_rmse / num_validation_files
    print(f"avg_val_rmse: {avg_val_rmse}")   

    # estimate on test set
    test_set_estimations = []
    test_rmse = 0.0
    test_mae = 0.0
    test_mse = 0.0
    for X, y in lst_test_dfs:

        # get SoC [%]
        soc_ground_truth = X[:,-1].reshape(-1, 1)     # reshape to match shape of model output

        # get SoC from Coulomn Counting method
        coulomb_couting_output = coulomb_counting(X).reshape(-1, 1)    # reshape to match shape of model output
        
        X = X[:,features_indexes_for_ml]    # keep only relevant columns for ML

        y_pred = model.predict(X)
        
        # processing estimation 
        y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
        soc_estimated = coulomb_couting_output + y_pred    # y_pred is delta
        test_set_estimations.append(soc_estimated)    # store soc_estimated before scalling it for evaluation
        # print(soc_estimated)
        soc_estimated = scaler_soc.transform(soc_estimated.reshape(-1, 1))
        
        test_rmse += root_mean_squared_error(soc_ground_truth, soc_estimated)
        test_mse += mean_squared_error(soc_ground_truth, soc_estimated)
        test_mae += mean_absolute_error(soc_ground_truth, soc_estimated)
        
    avg_test_rmse = test_rmse / num_test_files
    avg_test_mse = test_mse / num_test_files
    avg_test_mae = test_mae / num_test_files
    
    print(f"avg_test_rmse: {avg_test_rmse}")  
    print(f"avg_test_mse: {avg_test_mse}")  
    print(f"avg_test_mae: {avg_test_mae}")  


    # add predictions to data
    files_data = [] 
    for index in range(len(lst_test_dfs)):
        X = lst_test_dfs[index][0]
        y = lst_test_dfs[index][1]
        file_data = [] 
        
        # combine X, y and  data to have everything in one list
        X_data = scaler_X.inverse_transform(X)
        y_data = scaler_y.inverse_transform(y)
        X_and_y = [list(l1) + list(l2) for l1, l2 in zip(X_data, y_data)]
    
        file_data.extend(X_and_y)
    
        file_data = np.array(file_data)
        file_predictions_array = np.array(test_set_estimations[index])
        file_data = [list(l1) + list(l2) for l1, l2 in zip(file_data, file_predictions_array)]
        
        files_data.append(file_data)
    
    # create column name for model output
    estimation_column_name = f"""{'Physics' if use_physics else ''}{'-and-' if use_physics and use_ml else ''}{'XGBRegressor' if use_ml else ''}"""
    
    # store results
    # Create folder (if it doesn't exist) to store test predictions and X and y features
    output_folder_test_files = f"{output_folder}/test_data_and_model_output"
    os.makedirs(output_folder_test_files, exist_ok=True)
    for idx_file_data in range(len(files_data)):
        file_data = files_data[idx_file_data]
    
        # create df
        columns = list(X_column_names.copy())
    
        columns.extend(["Delta", estimation_column_name])
    
        # print(columns)
        
        # columns = ["Time [s]", "SoC_0", "Q_rated", "Battery Voltage [V]", "Battery Current [A]", "Time_difference", "SoC [%]", "Estimated SoC (Transformer model)"]
        df = pd.DataFrame(file_data, columns=columns)
    
        # save in created folder
        output_file = f"file-{idx_file_data}.csv"
        output_file = os.path.join(output_folder_test_files,output_file)
        df.to_csv(output_file, index=True, sep=";", encoding="ISO-8859-2")

avg_val_rmse: 0.038409681504841295
avg_test_rmse: 0.03804833624446123
avg_test_mse: 0.0018167981355072076
avg_test_mae: 0.03272370630175061


In [55]:
# Physics only model
if use_physics and not use_ml:

    # evaluation
    val_rmse = 0.0
    for X, y in lst_val_data:

        # get SoC [%]
        soc_ground_truth = X[:,-1].reshape(-1, 1)     # reshape to match shape of model output

        # get SoC from Coulomn Counting method
        coulomb_couting_output = coulomb_counting(X).reshape(-1, 1)    # reshape to match shape of model output

        # scale ground truth to calculate metric(s) like with transformer models.
        coulomb_couting_output = scaler_soc.transform(coulomb_couting_output)
        
        val_rmse += root_mean_squared_error(soc_ground_truth, coulomb_couting_output)
        # print(val_rmse)
        
    avg_val_rmse = val_rmse / num_validation_files
    print(f"avg_val_rmse: {avg_val_rmse}")  

    # estimate on test set
    test_set_estimations = []
    test_rmse = 0.0
    test_mae = 0.0
    test_mse = 0.0
    for X, y in lst_test_dfs:

        # get SoC [%]
        soc_ground_truth = X[:,-1].reshape(-1, 1)     # reshape to match shape of model output

        # get SoC from Coulomn Counting method
        coulomb_couting_output = coulomb_counting(X).reshape(-1, 1)    # reshape to match shape of model output

        test_set_estimations.append(coulomb_couting_output)    # save estimation before scalling it to calculate metrics

        coulomb_couting_output = scaler_soc.transform(coulomb_couting_output)    # scale the output from Coulomb Counting
        
        test_rmse += root_mean_squared_error(soc_ground_truth, coulomb_couting_output)
        test_mse += mean_squared_error(soc_ground_truth, coulomb_couting_output)
        test_mae += mean_absolute_error(soc_ground_truth, coulomb_couting_output)
        
    avg_test_rmse = test_rmse / num_test_files
    avg_test_mse = test_mse / num_test_files
    avg_test_mae = test_mae / num_test_files
    
    print(f"avg_test_rmse: {avg_test_rmse}")  
    print(f"avg_test_mse: {avg_test_mse}")  
    print(f"avg_test_mae: {avg_test_mae}")   

    # add predictions to data
    files_data = [] 
    for index in range(len(lst_test_dfs)):
        X = lst_test_dfs[index][0]
        y = lst_test_dfs[index][1]
        file_data = [] 
        
        # combine X, y and  data to have everything in one list
        X_data = scaler_X.inverse_transform(X)
        y_data = scaler_y.inverse_transform(y)
        X_and_y = [list(l1) + list(l2) for l1, l2 in zip(X_data, y_data)]
    
        file_data.extend(X_and_y)
    
        file_data = np.array(file_data)
        file_predictions_array = np.array(test_set_estimations[index])
        file_data = [list(l1) + list(l2) for l1, l2 in zip(file_data, file_predictions_array)]
        
        files_data.append(file_data)
    
    # create column name for model output
    estimation_column_name = f"""{'Physics' if use_physics else ''}{'-and-' if use_physics and use_ml else ''}{'XGBRegressor' if use_ml else ''}"""
    
    # store results
    # Create folder (if it doesn't exist) to store test predictions and X and y features
    output_folder_test_files = f"{output_folder}/test_data_and_model_output"
    os.makedirs(output_folder_test_files, exist_ok=True)
    for idx_file_data in range(len(files_data)):
        file_data = files_data[idx_file_data]
    
        # create df
        columns = list(X_column_names.copy())
    
        columns.extend(["Delta", estimation_column_name])
    
        # print(columns)
        
        # columns = ["Time [s]", "SoC_0", "Q_rated", "Battery Voltage [V]", "Battery Current [A]", "Time_difference", "SoC [%]", "Estimated SoC (Transformer model)"]
        df = pd.DataFrame(file_data, columns=columns)
    
        # save in created folder
        output_file = f"file-{idx_file_data}.csv"
        output_file = os.path.join(output_folder_test_files,output_file)
        df.to_csv(output_file, index=True, sep=";", encoding="ISO-8859-2")

In [56]:
lst_train_data[0][:,features_indexes_for_ml]

array([[-1.32002637,  0.98255685,  1.24208573,  0.36720228,  0.49437417],
       [-1.31989889,  0.98255685,  1.24208573,  0.36696169,  0.49437417],
       [-1.31977142,  0.98255685,  1.24208573,  0.36575878,  0.49437417],
       ...,
       [ 0.09366603,  0.60842274,  0.11554667,  0.3036883 , -1.60589563],
       [ 0.0937935 ,  0.60842274,  0.1107853 ,  0.31571746, -1.60589563],
       [ 0.09392098,  0.60842274,  0.10888076,  0.32149146, -1.60589563]])

In [57]:
lst_train_data[0][:, -1] 

array([1.44532151, 1.44532151, 1.44532151, ..., 0.04857047, 0.04857047,
       0.04857047])

In [58]:
# ML only model
if (use_ml and not use_physics):
    # train ml + physics.
    X = X = lst_train_data[0][:,features_indexes_for_ml]    # keep only relevant columns for ML
    y = lst_train_data[0][:, -1]   # y is in this case SoC directly
    
    model = XGBRegressor(**params, random_state=2025)
    model.fit(X, y)

    # save model
    model.save_model(f"{output_folder}/trained_model.json")

    # evaluation
    val_rmse = 0.0
    for X, y in lst_val_data:

        # get SoC [%]
        soc_ground_truth = X[:,-1].reshape(-1, 1)     # reshape to match shape of model output

        X = X[:,features_indexes_for_ml]    # keep only relevant columns for ML

        y_pred = model.predict(X)
        
        val_rmse += root_mean_squared_error(soc_ground_truth, y_pred)
        # print(val_rmse)
        
    avg_val_rmse = val_rmse / num_validation_files
    print(f"avg_val_rmse: {avg_val_rmse}")   

    # estimate on test set
    test_set_estimations = []
    test_rmse = 0.0
    test_mae = 0.0
    test_mse = 0.0
    for X, y in lst_test_dfs:

        # get SoC [%]
        soc_ground_truth = X[:,-1].reshape(-1, 1)     # reshape to match shape of model output
        
        X = X[:,features_indexes_for_ml]    # keep only relevant columns for ML

        y_pred = model.predict(X)
        
        # processing estimation 
        y_pred = scaler_soc.inverse_transform(y_pred.reshape(-1, 1))
        test_set_estimations.append(y_pred)    # store soc_estimated before scalling it for evaluation
        y_pred = scaler_soc.transform(y_pred)    #  scale the output again to calculate the metrics

        test_rmse += root_mean_squared_error(soc_ground_truth, y_pred)
        test_mse += mean_squared_error(soc_ground_truth, y_pred)
        test_mae += mean_absolute_error(soc_ground_truth, y_pred)
        
    avg_test_rmse = test_rmse / num_test_files
    avg_test_mse = test_mse / num_test_files
    avg_test_mae = test_mae / num_test_files
    
    print(f"avg_test_rmse: {avg_test_rmse}")  
    print(f"avg_test_mse: {avg_test_mse}")  
    print(f"avg_test_mae: {avg_test_mae}")  


    # add predictions to data
    files_data = [] 
    for index in range(len(lst_test_dfs)):
        X = lst_test_dfs[index][0]
        y = lst_test_dfs[index][1]
        file_data = [] 
        
        # combine X, y and  data to have everything in one list
        X_data = scaler_X.inverse_transform(X)
        y_data = scaler_soc.inverse_transform(y)
        X_and_y = [list(l1) + list(l2) for l1, l2 in zip(X_data, y_data)]
    
        file_data.extend(X_and_y)
    
        file_data = np.array(file_data)
        file_predictions_array = np.array(test_set_estimations[index])
        file_data = [list(l1) + list(l2) for l1, l2 in zip(file_data, file_predictions_array)]
        
        files_data.append(file_data)
    
    # create column name for model output
    estimation_column_name = f"""{'Physics' if use_physics else ''}{'-and-' if use_physics and use_ml else ''}{'XGBRegressor' if use_ml else ''}"""
    
    # store results
    # Create folder (if it doesn't exist) to store test predictions and X and y features
    output_folder_test_files = f"{output_folder}/test_data_and_model_output"
    os.makedirs(output_folder_test_files, exist_ok=True)
    for idx_file_data in range(len(files_data)):
        file_data = files_data[idx_file_data]
    
        # create df
        columns = list(X_column_names.copy())
    
        columns.extend(["Delta", estimation_column_name])
    
        # print(columns)
        
        # columns = ["Time [s]", "SoC_0", "Q_rated", "Battery Voltage [V]", "Battery Current [A]", "Time_difference", "SoC [%]", "Estimated SoC (Transformer model)"]
        df = pd.DataFrame(file_data, columns=columns)
    
        # save in created folder
        output_file = f"file-{idx_file_data}.csv"
        output_file = os.path.join(output_folder_test_files,output_file)
        df.to_csv(output_file, index=True, sep=";", encoding="ISO-8859-2")   

In [59]:
# export metrics
metrics = {
    "validation set RMSE": avg_val_rmse,
    "test set loss (RMSE)": avg_test_rmse,
    "test set loss (MSE)": avg_test_mse,
    "test set loss (MAE)": avg_test_mae,
}

with open(f"{output_folder}/metrics.json", 'w') as file:
    json.dump(metrics, file)

In [60]:
# export hyperparameters and other relevant variables
hyperparameters_and_other_relevant_variables = {
    "model_name": model_name,
    "NORMALIZE_DATA": NORMALIZE_DATA,
    "ML_MODEL_USES_CALCULATED_COLUMNS": ML_MODEL_USES_CALCULATED_COLUMNS,
    "dataset_folder": dataset_folder,
    "required_columns": required_columns,
    "calculated_columns": calculated_columns,
    
    **params,

    "use_physics": use_physics,
    "use_ml": use_ml,

}


with open(f"{output_folder}/hyperparameters_and_relevant_variables.json", 'w') as file:
    json.dump(hyperparameters_and_other_relevant_variables, file)